In [1]:
from src.api import analysis as api
from src.app import App
import ipywidgets as widgets

In [2]:
app = App()
def log_fcn(*args):
    app.log_ctx.append_stdout(' '.join([str(arg) for arg in args]))
app.container

In [3]:
df_samples, df_counts, df_fpkm, df_fpkm_uq = api.get_data()
with app.log_context:
    print('data frame shape')
    print(df_samples.shape)

In [4]:
allowed_project_ids = list(df_samples['project id'].unique()[2:5])
df_samples, df_counts, df_fpkm, df_fpkm_uq = api.filter_data_by_column_value(df_samples, df_counts, df_fpkm, df_fpkm_uq, 'project id',      allowed_project_ids)

In [5]:
nstd_by_column = {
        'average base quality': 2, 
        'proportion_base_mismatch': 2,
        'proportion_reads_mapped': 2
}

df_samples, df_counts, df_fpkm, df_fpkm_uq = api.apply_cutoffs(df_samples, df_counts, df_fpkm, df_fpkm_uq, nstd_by_column)

In [6]:
with app.log_context:
    print('filtered frame shape')
    print(df_samples.shape)

In [7]:
with app.eda_ctx:
    display(widgets.HTML('<h2>Perform exploratory data analysis on the filtered sample labels</h2>'))
    api.perform_eda(df_samples)

In [8]:
df_tpm = api.calculate_tpm(df_counts)

In [9]:
with app.eda_ctx:
    display(widgets.HTML('<h2>Create and plot PCA analysis using the variance-stabilizing-transformed data from DESeq2</h2>'))
    api.perform_tsne(df_tpm, df_samples)

In [10]:
with app.eda_ctx:
    display(widgets.HTML('<h2>Create and plot tSNE analysis using the variance-stabilizing-transformed data from DESeq2</h2>'))
    api.perform_pca(df_tpm, df_samples)

In [11]:
with app.log_context:
    print('unique sample types')
    print(df_samples['sample type'].unique())

In [12]:
with app.log_context:
    print(df_samples.shape, df_counts.shape, df_fpkm.shape, df_fpkm_uq.shape)

In [13]:
with app.eda_ctx:
    display(widgets.HTML('<h2>Create and plot PCA autoencoder analysis using the variance-stabilizing-transformed data from DESeq2</h2>'))
    api.run_autoencoder(df_tpm, df_samples, dim_reduction_method='PCA', n_components=10)

In [14]:
with app.eda_ctx:
    display(widgets.HTML('<h2>Create and plot tSNE autoencoder analysis using the variance-stabilizing-transformed data from DESeq2</h2>'))
    api.run_autoencoder(df_tpm, df_samples, dim_reduction_method='tSNE', n_components=2)

In [15]:
from keras.models import load_model
encoder = load_model(api.AUTO_ENCODER_MODEL_PATH)

In [16]:
encoded_out = encoder.predict(df_tpm)
encoded_out

array([[0.00166219, 0.00165141, 0.0019213 , ..., 0.00146592, 0.00223878,
        0.00157726],
       [0.00131869, 0.00128186, 0.0016329 , ..., 0.00123847, 0.00180557,
        0.00144327],
       [0.00143281, 0.00154692, 0.00167009, ..., 0.00152338, 0.00199452,
        0.00146228],
       ...,
       [0.0012379 , 0.0014427 , 0.00159845, ..., 0.00139865, 0.00183082,
        0.00121108],
       [0.00134671, 0.00148332, 0.00171468, ..., 0.00134963, 0.00180474,
        0.00136027],
       [0.00152147, 0.00169381, 0.00186285, ..., 0.00156921, 0.00212482,
        0.00126401]], dtype=float32)

In [17]:
import pandas as pd
df = pd.DataFrame(encoded_out)

In [18]:
api.perform_pca(df, df_samples[api.LABEL_COLUMN_NAME].sort_index(), n_components=10)

KeyError: 'label 1'